## Midfielders Data Preprocessing

This notebook performs data preprocessing on historical data for midfielders in fantasy football.

It includes data loading, cleaning, feature engineering, and outlier removal.

**Contents:**
1. Data Loading
2. Missing Value Handling
3. Data Type Conversion
4. Feature Engineering
5. Scaling
6. Outlier Removal
7. Data Export

In [1]:
# necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

1. Data Loading

In [2]:
# Load historical data for midfielders
historical_data = pd.read_csv('../data/cleaned/midfielder/midfielder_cleaned.csv')

# Display initial data sample for midfielders
display(historical_data.head())

,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,...,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,id,first_name,second_name,unique_id
0,2023/24,535818,50,50,97,2220,6,1,4,38,...,173.4,25,4.19,3.26,7.45,36.64,113,Simon,Adingra,535818_2023/24
1,2012/13,39155,60,56,93,2230,3,5,4,42,...,0.0,0,0.00,0.00,0.00,0.00,463,Adam,Lallana,39155_2012/13
2,2013/14,39155,60,78,178,3086,9,9,16,41,...,0.0,0,0.00,0.00,0.00,0.00,463,Adam,Lallana,39155_2013/14
3,2014/15,39155,85,83,93,1779,5,3,9,22,...,0.0,0,0.00,0.00,0.00,0.00,463,Adam,Lallana,39155_2014/15
4,2015/16,39155,75,73,109,2109,4,6,10,27,...,0.0,0,0.00,0.00,0.00,0.00,463,Adam,Lallana,39155_2015/16


2. Missing Value Handling

In [3]:
# Check for NaN and missing values
print("\nColumns with NaN or missing values:")
print(historical_data.isnull().sum()[historical_data.isnull().sum() > 0])

# Fill NaN and missing values with 0
historical_data = historical_data.fillna(0)

# Verify NaN and missing values have been filled
print("\nRemaining NaN or missing values after filling:")
print(historical_data.isnull().sum()[historical_data.isnull().sum() > 0])

# Display the first few rows to verify the changes
print("\nFirst few rows after filling NaN and missing values:")
print(historical_data.head())

# Additional check for any remaining non-numeric values that might be considered as missing
non_numeric_cols = historical_data.select_dtypes(exclude=[np.number]).columns
for col in non_numeric_cols:
    if historical_data[col].dtype == 'object':
        print(f"\nUnique values in {col} column:")
        print(historical_data[col].unique())
        # Replace empty strings or other non-numeric placeholders with 0
        historical_data[col] = historical_data[col].replace(['', 'N/A', 'None', '-'], 0)

print("\nDataFrame info after handling all types of missing values:")
historical_data.info()


# Display the first few rows to verify the changes
print("\nFirst few rows of the updated DataFrame:")
print(historical_data.head())




Columns with NaN or missing values:
Series([], dtype: int64)

Remaining NaN or missing values after filling:
Series([], dtype: int64)

First few rows after filling NaN and missing values:
  season_name  element_code  start_cost  end_cost  total_points  minutes  \
0     2023/24        535818          50        50            97     2220   
1     2012/13         39155          60        56            93     2230   
2     2013/14         39155          60        78           178     3086   
3     2014/15         39155          85        83            93     1779   
4     2015/16         39155          75        73           109     2109   

   goals_scored  assists  clean_sheets  goals_conceded  ...  ict_index  \
0             6        1             4              38  ...      173.4   
1             3        5             4              42  ...        0.0   
2             9        9            16              41  ...        0.0   
3             5        3             9              22  ..

3. Data Type Conversion

In [4]:
historical_data.dtypes

season_name                    object
element_code                    int64
start_cost                      int64
end_cost                        int64
total_points                    int64
minutes                         int64
goals_scored                    int64
assists                         int64
clean_sheets                    int64
goals_conceded                  int64
own_goals                       int64
penalties_saved                 int64
penalties_missed                int64
yellow_cards                    int64
red_cards                       int64
saves                           int64
bonus                           int64
bps                             int64
influence                     float64
creativity                    float64
threat                        float64
ict_index                     float64
starts                          int64
expected_goals                float64
expected_assists              float64
expected_goal_involvements    float64
expected_goa

In [5]:
# Ensure numerical columns are correctly typed
historical_data['end_cost'] = historical_data['end_cost'].astype(float) / 10  # Convert to float and scale cost
historical_data['start_cost'] = historical_data['start_cost'].astype(float) / 10  # Convert to float and scale cost

4. Feature Engineering

In [6]:
# Convert categorical columns to category types
historical_data['position'] = 'MID'  # All players are midfielders

# Create additional features
historical_data['points_per_game'] = historical_data['total_points'] / historical_data['minutes'].replace(0, np.nan)
historical_data['value_for_money'] = historical_data['total_points'] / historical_data['end_cost']

# Scale features if necessary (example using min-max scaling)
scaler = MinMaxScaler()
historical_data[['scaled_cost', 'scaled_points']] = scaler.fit_transform(historical_data[['end_cost', 'total_points']])

# Identify outliers using IQR method
Q1 = historical_data['total_points'].quantile(0.25)
Q3 = historical_data['total_points'].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for acceptable range
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
historical_data = historical_data[(historical_data['total_points'] >= lower_bound) & (historical_data['total_points'] <= upper_bound)]

# Save the cleaned and processed historical data
historical_data.to_csv('../data/processed/midfielders_data_processed.csv', index=False)
print("Processed historical data for midfielders saved successfully!")

Processed historical data for midfielders saved successfully!
